# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.functional as F
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)
print('train dataset length: {}'.format(len(train_dataset)))
print('test dataset length: {}'.format(len(test_dataset)))
# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

train dataset length: 60000
test dataset length: 10000


Then, we define the model, object function and optimizer that we use to classify.

In [4]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self, num_classes=10):
        super(SimpleNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3, padding=1)
        self.conv2 = nn.Conv2d(6, 16, 3, padding=1)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 7 * 7, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        self.max_pool = nn.MaxPool2d((2, 2))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.max_pool(self.relu(self.conv1(x)))
        x = self.max_pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = SimpleNet()
print(model)

# TODO:define loss function and optimiter
device = 'cpu'
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

SimpleNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU(inplace)
)


Next, we can start to train and evaluate!

In [5]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_total = 0
    train_correct = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        # TODO:forward + backward + optimize
        optimizer.zero_grad()  # zero the gradient buffers
        outputs = model(images)
        loss = criterion(outputs, labels)
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        loss.backward()
        optimizer.step()
    test_total = 0
    test_correct = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()
    train_acc = train_correct / train_total
    test_acc = test_correct / test_total
    print('[epoch]: {} | [train_acc]: {} | [test_acc]: {}'.format(epoch, train_acc, test_acc))

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:02<00:00, 27.40it/s]


[epoch]: 0 | [train_acc]: 0.21053685897435898 | [test_acc]: 0.39242788461538464


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:02<00:00, 26.19it/s]


[epoch]: 1 | [train_acc]: 0.5418502938034188 | [test_acc]: 0.8117988782051282


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:02<00:00, 27.72it/s]


...

[epoch]: 7 | [train_acc]: 0.9602029914529915 | [test_acc]: 0.9672475961538461


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:03<00:00, 24.84it/s]


[epoch]: 8 | [train_acc]: 0.964309561965812 | [test_acc]: 0.9703525641025641


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:03<00:00, 25.52it/s]


[epoch]: 9 | [train_acc]: 0.9681323450854701 | [test_acc]: 0.9711538461538461


#### Q5:
Please print the training and testing accuracy.

In [7]:
print('Testing accuracy: %.2f%%' % (test_acc * 100))

Testing accuracy: 97.12%
